### 3. Bonus Task: Predict the Aggregate eCPM ( combining ( aggregating on ) both Amp and Non-Amp ) for 1 month.

In [8]:
import pandas as pd
import numpy as np
from math import ceil,floor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import warnings
from sklearn.cross_validation import train_test_split,cross_val_score
warnings.simplefilter('ignore', DeprecationWarning)
warnings.simplefilter('ignore', RuntimeWarning)

In [9]:
def week_of_month(dt):
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + first_day.weekday()
    return int(ceil(adjusted_dom/7.0))

In [10]:
df=pd.read_csv("DFP_solution.csv")

#Data pre-processing
df["DATE"]=pd.to_datetime(df["DATE"])
data=df[["DATE","Actual_eCPM","Tags_served","Impressions","Clicks","CTR"]].groupby(["DATE"],as_index=False).sum()
map_day={"Monday":1,"Tuesday":2,"Wednesday":3,"Thursday":4,"Friday":5,"Saturday":6,"Sunday":7}
data["Day_number"]=data["DATE"].apply(lambda x: map_day[x.date().strftime("%A")])
data["Week_number"]=data["DATE"].apply(lambda x: week_of_month(x))
data=data[['Actual_eCPM','Tags_served', 'Impressions', 'Clicks', 'CTR','Day_number', 'Week_number']]

In [11]:
#Feature and Target Variable
feature=data.iloc[:,1:]
target=data.iloc[:,0]

In [12]:
#Modelling
#Parameter are tuned using grid cv search
model=RandomForestRegressor(n_estimators=8,random_state=21,max_depth=8,min_samples_split=2,min_samples_leaf=1,n_jobs=4,bootstrap=False)
model.fit(feature,target)

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=8,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=8, n_jobs=4,
           oob_score=False, random_state=21, verbose=0, warm_start=False)

In [13]:
df=pd.read_csv("DFP_prediction.csv")
#Data pre-processing
df["DATE"]=pd.to_datetime(df["DATE"])
data=df[["DATE","Tags_served","Impressions","Clicks","CTR"]].groupby(["DATE"],as_index=False).sum()
map_day={"Monday":1,"Tuesday":2,"Wednesday":3,"Thursday":4,"Friday":5,"Saturday":6,"Sunday":7}
data["Day_number"]=data["DATE"].apply(lambda x: map_day[x.date().strftime("%A")])
data["Week_number"]=data["DATE"].apply(lambda x: week_of_month(x))
data=data[['Tags_served', 'Impressions', 'Clicks', 'CTR','Day_number', 'Week_number']]

In [14]:
predictions=model.predict(data)

In [15]:
df=pd.read_csv("sub.csv")
df["eCPM"]=predictions
df.to_csv("sub.csv",index=False)